In [1]:
from transformers import Qwen3VLForConditionalGeneration, AutoProcessor
import torch
import os
import pandas as pd
from datasets import load_dataset

SLURM_PATH = '/home/yandex/MLWG2025/amitr5'
CACHE_DIR = f'{SLURM_PATH}/tmp/hf_cache'  # Changed to /tmp to avoid quota issues

os.makedirs(CACHE_DIR, exist_ok=True)

if SLURM_PATH in os.getcwd():
    os.environ["PIP_PATH"] = f"{SLURM_PATH}/BaryGNN/anaconda3/envs/conf/bin/pip"
    os.environ["TEMP_DIR"] = CACHE_DIR
    os.environ["HF_HOME"] = CACHE_DIR
    os.environ["TRANSFORMERS_CACHE"] = CACHE_DIR
    os.environ["HF_DATASETS_CACHE"] = CACHE_DIR
    os.environ["HF_HUB_CACHE"] = CACHE_DIR
    os.environ["TMPDIR"] = CACHE_DIR
    # os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
import importlib
import sys

def reload(*module_names):
    for name in module_names:
        if name in sys.modules:
             importlib.reload(sys.modules[name])
             print(f"{name}- reloaded")
        else:
            print(f"{name}- NOT FOUND!")

In [36]:

import torch

import sys
sys.path.append('./Mind2Web/src') 

ds = load_dataset("osunlp/Multimodal-Mind2Web", cache_dir=CACHE_DIR)

print("Dataset splits:", list(ds.keys()))
# Access the test splits
test_domain_ds = ds['test_domain']
test_task_ds = ds['test_task']
test_website_ds = ds['test_website']

print(f"Number of samples in test_domain: {len(test_domain_ds)}")
print(f"Number of samples in test_task: {len(test_task_ds)}")
print(f"Number of samples in test_website: {len(test_website_ds)}")
print("Total number of test samples:", len(test_domain_ds) + len(test_task_ds) + len(test_website_ds))

Resolving data files:   0%|          | 0/27 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/23 [00:00<?, ?it/s]

Dataset splits: ['train', 'test_domain', 'test_task', 'test_website']
Number of samples in test_domain: 4060
Number of samples in test_task: 1339
Number of samples in test_website: 1019
Total number of test samples: 6418


In [3]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, T5Tokenizer
from typing import Dict, Any
import sys
sys.path.append('./Mind2Web/src') 
from action_prediction.metric import ActionEvaluatorGeneration, ActionEvaluatorMultiChoice

reload('action_prediction.dataloader')
from mind2web.dataloader import MultiChoiceDataset, get_data_split, subsample_by_annotation

split_file = "test_task"  # or "test_task" or "test_website"

candidate_results = pd.read_pickle(f"candidates/scores_{split_file}.pkl")
flattened = get_data_split(
    data_dir="osunlp/Multimodal-Mind2Web",
    split_file=split_file,
    candidate_results=candidate_results,
    cache_dir=CACHE_DIR
)

def tensorize_item(item: Dict[str, Any], device: str):
    """
    Convert the model_input dict returned by MultiChoiceDataset.__getitem__
    (lists of ints) into tensors appropriate for model.generate.
    """
    input_ids = torch.LongTensor(item["input_ids"]).unsqueeze(0).to(device)
    attention_mask = torch.LongTensor(item["attention_mask"]).unsqueeze(0).to(device)
    return {"input_ids": input_ids, "attention_mask": attention_mask}


# Load tokenizer and model
# tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False, cache_dir=CACHE_DIR)
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-xl", cache_dir=CACHE_DIR)

# Construct MultiChoiceDataset like evaluate.py does.
dataset = MultiChoiceDataset(
        flattened,
        tokenizer,
        num_candidates=5,
        max_context_len=512,
        # mode="generation",  # use generation formatting
        mode="multichoice",  # use multichoice formatting
    )

device = "cuda" if torch.cuda.is_available() else "cpu"
model_name = "osunlp/MindAct_ActionPrediction_flan-t5-xl" #"Qwen/Qwen-3.5-VL-Base"
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, cache_dir=CACHE_DIR)
model = model.to(device)
model.eval()



action_prediction.dataloader- reloaded


Resolving data files:   0%|          | 0/27 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 2048)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 2048)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=2048, out_features=2048, bias=False)
              (k): Linear(in_features=2048, out_features=2048, bias=False)
              (v): Linear(in_features=2048, out_features=2048, bias=False)
              (o): Linear(in_features=2048, out_features=2048, bias=False)
              (relative_attention_bias): Embedding(32, 32)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=2048, out_features=5120, bias=False)
              (wi_1): Linear(in_features=2048, out_features=5120, bias=False)
       

In [ ]:
reload('mind2web.dataloader')
from mind2web.dataloader import MultiChoiceDataset, get_data_split, subsample_by_annotation
from mind2web.dataloader import subsample_by_annotation
SEED = 42

cal_set, test_set = subsample_by_annotation(flattened, frac=0.05, seed=SEED)
cal_ds = MultiChoiceDataset(cal_set, tokenizer, num_candidates=5, max_context_len=512)
test_ds = MultiChoiceDataset(test_set, tokenizer, num_candidates=5, max_context_len=512)

mind2web.dataloader- reloaded


Filter:   0%|          | 0/1339 [00:00<?, ? examples/s]

/home/yandex/MLWG2025/amitr5/BaryGNN/anaconda3/envs/conf/lib/python3.13/site-packages/PIL/Image.py:3432: DecompressionBombWarning: Image size (125915272 pixels) exceeds limit of 89478485 pixels, could be decompression bomb DOS attack.
  warnings.warn(


Filter:   0%|          | 0/1339 [00:00<?, ? examples/s]

In [ ]:
import random
import numpy as np
import torch.nn.functional as F
from tqdm import tqdm
# random.seed(42)
# np.random.seed(42)


choices_to_token_ids = dataset.choices_token_ids_mapping()
outputs = []
all_scores = []  # Store scores for each sample
max_new_tokens = 1#50

for i, item in enumerate(tqdm(cal_ds, desc="Generating calibration set"), 1):
        model_input = tensorize_item(item, device)

        with torch.no_grad():
            out = model.generate(
                **model_input,
                eos_token_id=model.config.eos_token_id,
                max_new_tokens=max_new_tokens,
                return_dict_in_generate=True,
                output_scores=True,
            )

        decoded = tokenizer.batch_decode(out["sequences"], skip_special_tokens=True)[0]
        labels_tokens = item.get("labels")
       
        # Calculate choice probabilities
        logits = out["scores"][0][0]
        all_probs = F.softmax(logits, dim=-1)
        probs = all_probs[list(choices_to_token_ids.values())]
        choices_probs = dict(zip(choices_to_token_ids.keys(), probs.cpu().tolist()))
        labels = item.get("labels").strip()
        outputs.append(
            [
                  i, 
                  dataset.data[i].get("annotation_id"),
                  dataset.data[i].get("action_uid"),
                  decoded,
                  labels.split('.')[0],
                  labels, 
                  choices_probs,
                  choices_probs.get(decoded, 0)
            ]    
        )
        

cols = ["index", "annotation_id", "action_uid", "generated", "label",'label_text', "choices_probs", "prob"]
results_df = pd.DataFrame(outputs, columns=cols)
results_df.to_pickle("cal_results.pkl")

Generating calibration set: 100%|██████████| 11/11 [00:51<00:00,  4.69s/it]

Processed 11 samples total
------------------------------------------------------------


In [22]:
results_df.head()

,index,annotation_id,action_uid,generated,label,label_text,choices_probs,prob
0,1,91695df8-f256-47c9-8c37-06e8d0fc758f,ea2865e4-2858-478c-bf83-93d576cad774,B,B,B.\nAction: CLICK,"{'A': 0.00154191255569458, 'B': 0.998367965221...",0.998368
1,2,91695df8-f256-47c9-8c37-06e8d0fc758f,c70a5d9b-8777-4ff7-a17d-1f76012e14ba,B,B,B.\nAction: CLICK,"{'A': 0.11233486980199814, 'B': 0.887105524539...",0.887106
2,3,91695df8-f256-47c9-8c37-06e8d0fc758f,3e671043-cab2-4e44-a1ce-3ed9de91d16b,B,B,B.\nAction: TYPE\nValue: Dublin,"{'A': 0.12114264070987701, 'B': 0.878037214279...",0.878037
3,4,91695df8-f256-47c9-8c37-06e8d0fc758f,60888ab2-b605-4886-8dac-6ba72b6cde0a,F,F,F.\nAction: CLICK,"{'A': 0.00515311025083065, 'B': 6.382887931977...",0.994768
4,5,91695df8-f256-47c9-8c37-06e8d0fc758f,43809daf-0eb6-40ab-96df-0c2bf143e25c,B,B,B.\nAction: CLICK,"{'A': 0.011086024343967438, 'B': 0.98851072788...",0.988511


In [ ]:
results_df['correct'] = results_df['generated'] == results_df['label']
grouped = results_df.groupby('annotation_id')
print(np.all(results_df['correct'].values))
score_per_group = 1 - grouped['prob'].min()
score_per_group
alpha = 0.05

N = len(score_per_group)
k = np.ceil((N + 1) * alpha)
q_level = k/N



True


annotation_id
91695df8-f256-47c9-8c37-06e8d0fc758f    0.121963
Name: prob, dtype: float64

In [ ]:
# TODO : calc scores , plot dist , find threshold

dict_keys(['action_uid', 'raw_html', 'cleaned_html', 'operation', 'pos_candidates', 'neg_candidates', 'website', 'domain', 'subdomain', 'annotation_id', 'confirmed_task', 'screenshot', 'action_reprs', 'target_action_index', 'target_action_reprs'])

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

from mind2web.dataloader import MultiChoiceDataset, get_data_split
from mind2web.evauate import FastActionEvaluatorMultiChoice  # note the file name
# if installed as a package, you can also do: from mind2web.evauate import FastActionEvaluatorMultiChoice

# Load tokenizer/model
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to("cuda")

# Prepare data (adjust paths/splits as needed)
data_dir = "DATA_PATH"
split_file = "data/test_task/*.json"  # example split
data = get_data_split(data_dir, split_file)
dataset = MultiChoiceDataset(data, tokenizer, num_candidates=5, max_context_len=512, mode="multichoice")

# Run evaluation
evaluator = FastActionEvaluatorMultiChoice(tokenizer, max_context_len=512)
result = evaluator.evaluate_dataset(
    dataset,
    model,
    batch_size=8,
    top_k=50,
    output_path=None,  # set a folder path to save JSON outputs
    name="test_task",
    template=None,     # optionally a tuple (prefix_context, suffix_input)
    max_new_tokens=50,
)

print(result)


{0: '013781df-4391-4533-bcb1-15f6819064f6',
 132: 'ffc01d9d-bf28-4787-a42c-b92b44db3285',
 134: 'ffc01d9d-bf28-4787-a42c-b92b44db3285',
 137: 'ffc01d9d-bf28-4787-a42c-b92b44db3285',
 145: 'ffc01d9d-bf28-4787-a42c-b92b44db3285',
 178: '5cf2503b-9026-423a-882c-d19018f86fb2',
 273: '86f52e05-c352-4bab-a101-df2ddf566b57',
 280: '88b53da2-6476-4cef-9946-ce2904187f43',
 282: '88b53da2-6476-4cef-9946-ce2904187f43',
 291: '502a9d95-af93-4f9c-a017-7dcd36c64a5d',
 295: '63be6edd-ebf5-446f-a61e-7b9990c850ae',
 297: '63be6edd-ebf5-446f-a61e-7b9990c850ae',
 299: '79f72a57-2967-44b5-b0cd-2501ef64769b',
 331: '375bd12d-d44b-4453-b6e3-c0f1889ceb82',
 333: '375bd12d-d44b-4453-b6e3-c0f1889ceb82',
 356: '9aec1bbb-e7ac-45e4-bcbd-ec2cee25af1a',
 420: 'a4bfaf1d-ba19-4af0-9618-8ab467eb7b5a',
 429: 'd5602fc7-7ab7-441b-b9ca-03b64ae81261',
 431: 'd5602fc7-7ab7-441b-b9ca-03b64ae81261',
 437: 'de69733d-58cb-4118-a38f-ef684fc4d07d',
 439: 'de69733d-58cb-4118-a38f-ef684fc4d07d',
 510: 'e719ca92-3b81-4f40-b2d1-a3bf8

In [ ]:
import sys
import importlib

sys.path.append('./Mind2Web/src') 

# Force reload the metric module
if 'action_prediction.metric' in sys.modules:
    importlib.reload(sys.modules['action_prediction.metric'])

from action_prediction.metric import ActionEvaluatorGeneration, ActionEvaluatorMultiChoice

evaluator = ActionEvaluatorMultiChoice(tokenizer)
test_dataset = dataset
test_key = "task"
output_path = os.getcwd()
result = evaluator.evaluate_dataset(
    test_dataset,
    model,
    output_path=output_path,
    name=test_key,
    template=None, #lm_template,
    top_k=50#cfg.top_k,)
)


 12%|█▏        | 158/1339 [48:58<6:06:01, 18.60s/it, action_f1=0, element_acc=0]



KeyboardInterrupt: 

In [ ]:
# df = pd.DataFrame(test_task_ds)
# len(df.groupby("annotation_id"))
# subsample[0]['action_reprs']

# test_domain_ds = ds['test_domain']
# test_task_ds = ds['test_task']
# test_website_ds = ds['test_website']
count = 0
missing_count = 0
mulit = {}
missing = {}
for i, x in enumerate(test_task_ds):
    if len(x['pos_candidates']) > 1:
        count += 1
        mulit[i] = x['annotation_id']
    if len(x['pos_candidates']) == 0:
        # print('Missing pos_candidates:', x['annotation_id'])
        missing_count += 1
        missing[i] = x['annotation_id']

print("Number of samples with multiple positive candidates in test_website:", count)
print("Number of samples with missing positive candidates in test_website:", missing_count)

In [ ]:

import argparse
import json
import pathlib
import sys
from typing import Any, Dict

import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Import the repository's dataset utilities
from action_prediction.dataloader import MultiChoiceDataset, get_data_split  # type: ignore





def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("--model-name", default="google/flan-t5-base")
    parser.add_argument("--data-dir", required=True, help="data dir or dataset script used by get_data_split")
    parser.add_argument("--split-file", required=True, help="path to split json file or list accepted by get_data_split")
    parser.add_argument("--num-samples", type=int, default=10)
    parser.add_argument("--device", default="cuda" if torch.cuda.is_available() else "cpu")
    parser.add_argument("--out", default="multichoice_generations.json")
    parser.add_argument("--max-new-tokens", type=int, default=50)
    args = parser.parse_args()

    outputs = generate_with_dataset(
        model_name=args.model_name,
        data_dir=args.data_dir,
        split_file=args.split_file,
        num_samples=args.num_samples,
        device=args.device,
        max_new_tokens=args.max_new_tokens,
    )

    with open(args.out, "w") as f:
        json.dump(outputs, f, indent=2)
    print(f"Wrote {len(outputs)} generations to {args.out}")


if __name__ == "__main__":
    main()

In [4]:
# Group by annotation_id (this creates a GroupBy object for fast access)
grouped = df.groupby('annotation_id')

# Retrieve all rows for a specific annotation_id
ann_id = train_ds[0]["annotation_id"]
task_df = grouped.get_group(ann_id).sort_values('target_action_index')

print(f"Task: {task_df.iloc[0]['confirmed_task']}")
# Iterate and display (task_df is a DataFrame)
for _, ex in task_df.iterrows():
    print(
        f"step={int(ex['target_action_index']) + 1}/{len(task_df)} | op={ex['operation']} "
        f"| target_action={ex["target_action_reprs"]} | pos_candidates={len(ex['pos_candidates'])}, action_id={ex['action_id']}"
    )
    # display(train_ds[ex["action_id"]]["screenshot"])

Task: rent a car in Brooklyn - Central, NY on from April 9 to April 15.
step=1/7 | op={"original_op": "CLICK", "value": "", "op": "CLICK"} | target_action=[heading]  CAR -> CLICK | pos_candidates=1, action_id=0
step=2/7 | op={"original_op": "TYPE", "value": "Brooklyn Central", "op": "TYPE"} | target_action=[combobox]  Enter pick up city, airport name, or airport code. -> TYPE: Brooklyn Central | pos_candidates=1, action_id=1
step=3/7 | op={"original_op": "CLICK", "value": "", "op": "CLICK"} | target_action=[div]  Brooklyn - Central (New York), US -> CLICK | pos_candidates=1, action_id=2
step=4/7 | op={"original_op": "CLICK", "value": "", "op": "CLICK"} | target_action=[textbox]  Pickup -> CLICK | pos_candidates=1, action_id=3
step=5/7 | op={"original_op": "CLICK", "value": "", "op": "CLICK"} | target_action=[button]  Sunday, April 9, 2023 -> CLICK | pos_candidates=1, action_id=4
step=6/7 | op={"original_op": "CLICK", "value": "", "op": "CLICK"} | target_action=[button]  Saturday, April

In [12]:
import torch
import sys
import os

# Add Mind2Web to path so we can import from it
sys.path.append('./Mind2Web/src')

from candidate_generation.dataloader import CandidateRankDataset, get_data_split
from candidate_generation.metric import CERerankingEvaluator
from candidate_generation.model import CrossEncoder
from torch.utils.data import DataLoader

eval_data = train_ds
batch_size = 350
max_seq_length = 512

eval_evaluator = CERerankingEvaluator(
    eval_data,
    k=50,
    max_neg=-1,
    batch_size=batch_size,
    name="train",
)

# Use the model path for the CrossEncoder (like in evaluate.py)
model_path = "osunlp/MindAct_CandidateGeneration_deberta-v3-base"
model = CrossEncoder(
    model_path,
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    num_labels=1,
    max_length=max_seq_length,
)

# Evaluate the model
print("Running evaluation...")
eval_evaluator(model, output_path="./output")
print("Evaluation completed!")
print("Results saved to: ./output")

ModuleNotFoundError: No module named 'dataloader'

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Download the MindAct Candidate Generation model
print("Downloading MindAct Candidate Generation model...")
model_name = "osunlp/MindAct_CandidateGeneration_deberta-v3-base"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=CACHE_DIR)

# Load model
candidate_model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    cache_dir=CACHE_DIR,
    torch_dtype=torch.float16,  # Use float16 for efficiency
    device_map="auto"  # Automatically handle device placement
)

print(f"Model {model_name} downloaded and loaded successfully!")
print(f"Model type: {type(candidate_model)}")
print(f"Number of parameters: {sum(p.numel() for p in candidate_model.parameters()):,}")